In [1]:
import seldon.pipeline.auto_transforms as auto
import seldon.pipeline.util as sutl
import seldon.pipeline.cross_validation as cf
from sklearn.pipeline import Pipeline
import pandas as pd
from seldon import xgb
import seldon.pipeline.basic_transforms as bt
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data",
                 header=None,names=["A1","A2","A3","A4","A5","A6","A7","A8","A9","A10","A11","A12","A13","A14","A15","target"])

/home/clive/tools/scikit-learn/sklearn/cross_validation.py:42: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,target
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [3]:
tTargetId = bt.Feature_id_transform(zero_based=True,input_feature="target",output_feature="targetId")
t_auto = auto.Auto_transform(exclude=["target","targetId"])
xg = xgb.XGBoostClassifier(target="targetId",excluded=["target"])
cv = cf.Seldon_KFold(xg,10)
transformers = [("tId",tTargetId),("tAuto",t_auto),("xgb",cv)]
p = Pipeline(transformers)
p.fit_transform(df)
print "avg cross validation accuracy ",cv.get_score()

avg cross validation accuracy  0.860869565217


In [4]:
from seldon.pipeline import bayes_optimize as bopt
e_opt = bopt.BayesOptimizer(xg,{'learning_rate': (0.0001, 0.1),'n_estimators': (10,1000),'max_depth':(3,20)},
                            param_int=['n_estimators','max_depth'],cv_folds=5)
transformers = [("tId",tTargetId),("tAuto",t_auto),("bopt",e_opt)]
p = Pipeline(transformers)
p.fit_transform(df)
print "Best accuracy is ",e_opt.get_best_score()

Initializing function at point:  {'n_estimators': 911.78093095347504, 'learning_rate': 0.058005156981055211, 'max_depth': 11.087767061828799} | result: 0.833333
Initializing function at point:  {'n_estimators': 809.62014847325509, 'learning_rate': 0.059786507579839755, 'max_depth': 18.652239686420895} | result: 0.821739
Initializing function at point:  {'n_estimators': 234.61512357766219, 'learning_rate': 0.026478710189946773, 'max_depth': 6.368707132444162} | result: 0.791304
Initializing function at point:  {'n_estimators': 990.19559680689167, 'learning_rate': 0.070816139750618817, 'max_depth': 12.430558275918466} | result: 0.826087
Initializing function at point:  {'n_estimators': 120.48141815266844, 'learning_rate': 0.062368361965343309, 'max_depth': 8.3352903838553125} | result: 0.781159
Iteration:   1 | Last sampled value:    0.776812 | with parameters:  {'n_estimators': 1000.0, 'learning_rate': 0.034517538483140806, 'max_depth': 7.8948236756820274}
               | Current maxim